# a01. Model Register in SAS
## How does a Model Developer Register Models to the Central Model Repository

---

## Table of Content
1. [訓練模型](#1.-訓練模型)<br>
2. [註冊模型](#2.-註冊模型)<br>
2-1. 新增連線<br>
2-2. 新增模型專案<br>
2-3. 更新模型專案的屬性<br>
2-4. 新增模型<br>
2-5. 新增模型檔案<br>
2-6. 新增模型的套件需求<br>
2-7. 簡易註冊模型 (Optional)<br>
3. [模型版本控制](#3.-模型版本控制)<br>
3-1. 新增模型的版本號碼<br>
3-2. 更新模型的屬性<br>
3-3. 確認模型的最新版本和屬性<br>



## Reference
1. [Python sasctl Package - Overview](https://github.com/sassoftware/python-sasctl)
2. [Python sasctl Package - User Guide](https://sassoftware.github.io/python-sasctl/index.html)
3. [SAS Model Manager - REST APIs](https://developer.sas.com/apis/rest/DecisionManagement/)
4. [SAS Model Manager - REST APIs Examples](https://github.com/sassoftware/devsascom-rest-api-samples/tree/master/DecisionManagement)

---
# 1. 訓練模型
[Top](#Table-of-Content)

In [1]:
import pandas as pd
import pickle
import os
import sys
from datetime import datetime
from pprint import pprint

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

In [2]:
%%time

column_type = {'LOAN':'float32', 'MORTDUE':'float32', 'VALUE':'float32', 'YOJ':'float32', 'DEROG':'float32', 
               'DELINQ':'float32', 'CLAGE':'float32', 'NINQ':'float32', 'CLNO':'float32', 'DEBTINC':'float32'}

data = pd.read_csv('https://support.sas.com/documentation/onlinedoc/viya/exampledatasets/hmeq.csv', dtype = column_type)

del data["REASON"]
del data["JOB"]

y = data['BAD']
X = data.iloc[:,1:]
X = X.fillna(0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 2021)

model_dt = tree.DecisionTreeClassifier()
model_dt.fit(X_train, y_train)

Wall time: 3.01 s


DecisionTreeClassifier()

### <font color='blue'>學員操作</font>

In [3]:
model_gb = GradientBoostingClassifier()
model_gb.fit(X_train, y_train)

GradientBoostingClassifier()

---
# 2. 註冊模型
[Top](#Table-of-Content)

### 2-1. 新增連線 (Connecting to SAS MM)

In [4]:
#SAS API
import sasctl
from sasctl import Session, register_model
from sasctl.services import folders, files, model_repository, projects

sas_session = Session('10.249.6.39', 'modelDeveloper', 'demopw', verify_ssl = False)

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.249.6.39'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


### 2-2. 新增模型專案

In [6]:
myProject_name = 'hmeq_project'

myRepository = model_repository.get_repository("Public")

input_variable_dict = [{'name': v, 'role': 'input', 'type': 'decimal', 'length': 8} for v in X.columns]
output_variable_dict = [{'name':'BAD', 'role':'output', 'type':'binary', 'length': 2, 'description': 'Put your comments here.'}]
# The role of the variable. Valid values are: input, output
# The type of variable. Valid values are: boolean, character, date, datetime, decimal, integer, binary, varying-length binary
# The measurement level of the variable. Valid values are: binary, interval, nominal, ordinal

if not model_repository.list_projects(filter= f'eq(name, {myProject_name})'):
    myProject = model_repository.create_project(
        project = {
            'name': myProject_name,
            'description': 'to demostrate management on Python models in SAS MM',
            'function': 'Classification',
            'variables': input_variable_dict + output_variable_dict
        },
        repository = myRepository
    )  

# 顯示模型專案的所有屬性
pprint(myProject)

{'candidateChampionHonored': False,
 'challengerModels': [],
 'createdBy': 'modelDeveloper',
 'creationTimeStamp': '2021-03-05T06:21:11.921Z',
 'description': 'to demostrate management on Python models in SAS MM',
 'folderId': 'dc52a9ea-8353-4143-b0bf-e89f3d9ec23a',
 'function': 'Classification',
 'globalTags': [],
 'id': '8373e5fc-ca71-4841-af2e-f77bdeca6ea5',
 'latestVersion': 'Version 1',
 'links': [{'href': '/modelRepository/projects',
            'itemType': 'application/vnd.sas.models.project.summary',
            'method': 'GET',
            'rel': 'up',
            'type': 'application/vnd.sas.collection',
            'uri': '/modelRepository/projects'},
           {'href': '/modelRepository/projects/8373e5fc-ca71-4841-af2e-f77bdeca6ea5',
            'method': 'GET',
            'rel': 'self',
            'type': 'application/vnd.sas.models.project',
            'uri': '/modelRepository/projects/8373e5fc-ca71-4841-af2e-f77bdeca6ea5'},
           {'href': '/modelRepository/proje

In [7]:
print(f'My project ID: \033[94m{myProject.get("id")}\033[0;0m')

My project ID: 8373e5fc-ca71-4841-af2e-f77bdeca6ea5


### 2-3. 更新模型專案的屬性

In [8]:
print(f'Target Variable: \033[94m{myProject.get("targetVariable")}\033[0;0m')
print(f'Class Target Values: \033[94m{myProject.get("classTargetValues")}\033[0;0m')
print(f'Target Event Value: \033[94m{myProject.get("targetEventValue")}\033[0;0m')

Target Variable: None
Class Target Values: None
Target Event Value: None


In [9]:
myProject['targetVariable'] = 'bad'
myProject['classTargetValues'] = 'bad,good'
myProject['targetEventValue'] = 'bad'

model_repository.update_project(myProject)

print(f'Target Variable: \033[94m{myProject.get("targetVariable")}\033[0;0m')
print(f'Class Target Values: \033[94m{myProject.get("classTargetValues")}\033[0;0m')
print(f'Target Event Value: \033[94m{myProject.get("targetEventValue")}\033[0;0m')

Target Variable: bad
Class Target Values: bad,good
Target Event Value: bad


### 2-4. 新增模型

In [10]:
myModel_name = 'Sklearn Decision Tree'

myModel_1 = model_repository.create_model(
    project = myProject,
    
    model = {
        'name': myModel_name,
        'trainCodeType': 'Python',
        'scoreCodeType': 'Python'
    },
    modeler = 'frank',    
    description = 'Manually create model artifacts',
    
    function = "Classification",
    algorithm = 'Decision Tree',
    tool = f'Python {sys.version_info.major}',
    is_challenger = True,
    
    input_variables = input_variable_dict,
    output_variables = output_variable_dict,
    
    properties = {
        'gitlab-access-token': 'ynJH8ppLyiB_2L5sT2NY',
        'gitlab-project-id': '24272994',
        'author_email': 'fkl@sas.com',
        'author_name': 'fkl'  
    }
)

# 顯示模型的所有屬性
pprint(myModel_1)

{'algorithm': 'Decision Tree',
 'candidateChampion': False,
 'createdBy': 'modelDeveloper',
 'creationTimeStamp': '2021-03-05T06:22:07.277Z',
 'description': 'Manually create model artifacts',
 'folderId': 'dc52a9ea-8353-4143-b0bf-e89f3d9ec23a',
 'function': 'Classification',
 'globalTags': [],
 'id': '4c9e48c6-83fe-496c-9acd-692576f48b29',
 'immutable': False,
 'indirectFolderId': 'dc52a9ea-8353-4143-b0bf-e89f3d9ec23a',
 'inputVariables': [{'createdBy': 'modelDeveloper',
                     'creationTimeStamp': '2021-03-05T06:22:07.388Z',
                     'id': 'db133fab-9f61-407e-b9f1-d7938d94bf84',
                     'length': 8,
                     'links': [{'href': '/modelRepository/models/4c9e48c6-83fe-496c-9acd-692576f48b29/variables/db133fab-9f61-407e-b9f1-d7938d94bf84',
                                'itemType': 'application/vnd.sas.models.variable',
                                'method': 'GET',
                                'rel': 'up',
                        

In [22]:
print(f'Sklearn Decision Tree model ID: \033[94m{myModel_1.get("id")}\033[0;0m')

Sklearn Decision Tree model ID: 4c9e48c6-83fe-496c-9acd-692576f48b29


### 2-5. 新增模型檔案 (以.pkl檔案為例)

In [11]:
with open('./model_dt.pkl', 'wb') as file:
    pickle.dump(model_dt, file)
    
myModel_pkl = open('./model_dt.pkl', 'rb')



myModelFile_1 = model_repository.add_model_content(
    model = myModel_1,
    file = myModel_pkl,
    name = 'model_dt.pkl',
    role = 'Python pickle'
)

# 顯示模型Pickle檔案的所有屬性
pprint(myModelFile_1)

{'createdBy': 'modelDeveloper',
 'creationTimeStamp': '2021-03-05T06:22:11.352Z',
 'fileUri': '/files/files/a3286458-7988-4de6-8f2c-8cc99fb650ad',
 'id': '57cd7ac4-f6a0-4e7d-92a0-d7d246f52557',
 'links': [{'href': '/modelRepository/models/4c9e48c6-83fe-496c-9acd-692576f48b29/contents/57cd7ac4-f6a0-4e7d-92a0-d7d246f52557',
            'method': 'GET',
            'rel': 'self',
            'type': 'application/vnd.sas.models.model.file',
            'uri': '/modelRepository/models/4c9e48c6-83fe-496c-9acd-692576f48b29/contents/57cd7ac4-f6a0-4e7d-92a0-d7d246f52557'},
           {'href': '/modelRepository/models/4c9e48c6-83fe-496c-9acd-692576f48b29/contents/57cd7ac4-f6a0-4e7d-92a0-d7d246f52557',
            'method': 'PUT',
            'rel': 'update',
            'type': 'application/vnd.sas.models.model.file',
            'uri': '/modelRepository/models/4c9e48c6-83fe-496c-9acd-692576f48b29/contents/57cd7ac4-f6a0-4e7d-92a0-d7d246f52557'},
           {'href': '/modelRepository/models/4c9e4

In [ ]:
print(f'model_dt.pkl file\'s content ID: {myModelFile_1.get("id")}')

### 2-6. 新增模型的套件需求

In [12]:
import pkg_resources

working_set = pkg_resources.working_set
list_package = sorted(['%s==%s' % (i.key, i.version) for i in working_set])

list_package_df = pd.DataFrame(list_package, columns = ['package'])
list_package_df.to_csv(f'./requirements.txt', index = False, header = False)

requirements_txt = open(f'./requirements.txt', 'r')

# os.system('powershell.exe pip freeze > ./requirements.txt')
# requirements_txt = open('./requirements.txt', 'r')       


myModelFile_2 = model_repository.add_model_content(
    model = myModel_1, 
    file = requirements_txt, 
    name = 'requirements.txt', 
    role = 'Requirements'
)

# 顯示該檔案的所有屬性
pprint(myModelFile_2)

{'createdBy': 'modelDeveloper',
 'creationTimeStamp': '2021-03-05T06:22:15.680Z',
 'fileUri': '/files/files/0ce76f51-7008-469e-a04f-f03abf20cec7',
 'id': 'a4667441-94ef-43c3-8b2f-0819851d6058',
 'links': [{'href': '/modelRepository/models/4c9e48c6-83fe-496c-9acd-692576f48b29/contents/a4667441-94ef-43c3-8b2f-0819851d6058',
            'method': 'GET',
            'rel': 'self',
            'type': 'application/vnd.sas.models.model.file',
            'uri': '/modelRepository/models/4c9e48c6-83fe-496c-9acd-692576f48b29/contents/a4667441-94ef-43c3-8b2f-0819851d6058'},
           {'href': '/modelRepository/models/4c9e48c6-83fe-496c-9acd-692576f48b29/contents/a4667441-94ef-43c3-8b2f-0819851d6058',
            'method': 'PUT',
            'rel': 'update',
            'type': 'application/vnd.sas.models.model.file',
            'uri': '/modelRepository/models/4c9e48c6-83fe-496c-9acd-692576f48b29/contents/a4667441-94ef-43c3-8b2f-0819851d6058'},
           {'href': '/modelRepository/models/4c9e4

In [ ]:
print(f'requirements.txt file\'s content ID: {myModelFile_2.get("id")}')

### <font color='blue'>學員操作</font>

In [13]:
yourModel = model_repository.create_model(
    project = myProject,
    
    model = {
        'name':'Sklearn Gradient Boosting',
        'trainCodeType':'Python',
        'scoreCodeType':'Python'
    },
    modeler = 'student',
    description = 'Manually create model artifacts',
    
    function = "Classification",
    algorithm = 'Gradient Boosting',
    tool = f'Python {sys.version_info.major}',
    is_challenger = True,
    
    input_variables = input_variable_dict,
    output_variables = output_variable_dict,
    
    properties = {
        'gitlab-access-token': 'ynJH8ppLyiB_2L5sT2NY',
        'gitlab-project-id': '24272994',
        'author_email': 'fkl@sas.com',
        'author_name': 'fkl'  
    }
)


#
with open('./model_gb.pkl', 'wb') as file:
    pickle.dump(model_gb, file)

yourModelFile_1 = model_repository.add_model_content(
    model = yourModel,
    file = open('./model_gb.pkl', 'rb'),
    name = 'model_gb.pkl',
    role = 'Python pickle'
)


#
os.system('powershell.exe pip freeze > ./requirements_student.txt')
requirements_student_txt = open('./requirements_student.txt', 'r')       


yourModelFile_2 = model_repository.add_model_content(
    model = yourModel, 
    file = requirements_student_txt, 
    name = 'requirements_student.txt', 
    role = 'Requirements'
)

# 顯示模型的所有屬性
pprint(yourModel)
print(f'\nSklearn Gradient Boosting model ID: {myModel_1.get("id")}')

{'algorithm': 'Gradient Boosting',
 'candidateChampion': False,
 'createdBy': 'modelDeveloper',
 'creationTimeStamp': '2021-03-05T06:22:20.210Z',
 'description': 'Manually create model artifacts',
 'folderId': 'dc52a9ea-8353-4143-b0bf-e89f3d9ec23a',
 'function': 'Classification',
 'globalTags': [],
 'id': 'd238c041-cc45-4681-929a-4d3f1cb08f4e',
 'immutable': False,
 'indirectFolderId': 'dc52a9ea-8353-4143-b0bf-e89f3d9ec23a',
 'inputVariables': [{'createdBy': 'modelDeveloper',
                     'creationTimeStamp': '2021-03-05T06:22:20.336Z',
                     'id': '5aece5f2-8cdd-49ee-b2a3-5e1531b74a76',
                     'length': 8,
                     'links': [{'href': '/modelRepository/models/d238c041-cc45-4681-929a-4d3f1cb08f4e/variables/5aece5f2-8cdd-49ee-b2a3-5e1531b74a76',
                                'itemType': 'application/vnd.sas.models.variable',
                                'method': 'GET',
                                'rel': 'up',
                    

### <font color='red'>2-7. 簡易註冊模型 (Optional)</font>

In [14]:
model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)

myModel_2 = register_model(
    project = myProject,
    name = "Sklearn Random Forest",
    model = model_rf,     
    input = data,
    record_packages = True # Capture Python packages registered in the environment
)

pprint(myModel_2)
print(f'\nSklearn Random Forest model ID: {myModel_2.get("id")}')

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.249.6.39'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.249.6.39'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'algorithm': 'Forest',
 'candidateChampion': False,
 'createdBy': 'modelDeveloper',
 'creationTimeStamp': '2021-03-05T06:22:28.564Z',
 'description': 'RandomForestClassifier()',
 'folderId': 'dc52a9ea-8353-4143-b0bf-e89f3d9ec23a',
 'function': 'classification',
 'globalTags': [],
 'id': '2c59d2f5-249a-474a-81dc-c34a2ef05234',
 'immutable': False,
 'indirectFolderId': 'dc52a9ea-8353-4143-b0bf-e89f3d9ec23a',
 'links': [{'href': '/modelRepository/models/2c59d2f5-249a-474a-81dc-c34a2ef05234',
            'method': 'GET',
            'rel': 'self',
            'type': 'application/vnd.sas.models.model',
            'uri': '/modelRepository/models/2c59d2f5-249a-474a-81dc-c34a2ef05234'},
           {'href': '/modelRepository/models/2c59d2f5-249a-474a-81dc-c34a2ef05234',
            'method': 'GET',
            'rel': 'alternate',
            'type': 'application/vnd.sas.models.model.summary',
            'uri': '/modelRepository/models/2c59d2f5-249a-474a-81dc-c34a2ef05234'},
           {'hre

# 3. 模型版本控制
[Top](#Table-of-Content)

### 3-1. 新增模型的版本號碼

In [30]:
desired_model_id = myModel_1.get('id')

desired_model = model_repository.get_model(item = desired_model_id)

desired_model_new = model_repository.create_model_version(model = desired_model, minor = False)

### 3-2. 更新模型的屬性

In [32]:
print(f'Target Variable: \033[94m{desired_model_new.get("targetVariable")}\033[0;0m')
print(f'Target Event Value: \033[94m{desired_model_new.get("targetEventValue")}\033[0;0m')
print(f'Modeler: \033[94m{desired_model_new.get("modeler")}\033[0;0m')

Target Variable: BADDDD
Target Event Value: None
Modeler: frank


In [33]:
desired_model_new['targetVariable'] = 'BAD'
desired_model_new['targetEventValue'] = 'bad'
desired_model_new['modeler'] = 'frank lin'

model_repository.update_model(desired_model_new)

print(f'Target Variable: \033[94m{desired_model_new.get("targetVariable")}\033[0;0m')
print(f'Target Event Value: \033[94m{desired_model_new.get("targetEventValue")}\033[0;0m')
print(f'Modeler: \033[94m{desired_model_new.get("modeler")}\033[0;0m')

Target Variable: BAD
Target Event Value: bad
Modeler: frank lin


### 3-3. 確認模型的最新版本和屬性

In [38]:
pprint(desired_model_new)

{'algorithm': 'Decision Tree',
 'candidateChampion': False,
 'createdBy': 'modelDeveloper',
 'creationTimeStamp': '2021-03-05T06:22:07.277Z',
 'description': 'Manually create model artifacts',
 'folderId': 'dc52a9ea-8353-4143-b0bf-e89f3d9ec23a',
 'function': 'Classification',
 'globalTags': [],
 'id': '4c9e48c6-83fe-496c-9acd-692576f48b29',
 'immutable': False,
 'indirectFolderId': 'dc52a9ea-8353-4143-b0bf-e89f3d9ec23a',
 'inputVariables': [{'createdBy': 'modelDeveloper',
                     'creationTimeStamp': '2021-03-05T06:22:07.388Z',
                     'id': 'd7578afd-1619-4731-af8a-4182b94c97f0',
                     'length': 8,
                     'links': [{'href': '/modelRepository/models/4c9e48c6-83fe-496c-9acd-692576f48b29/variables/d7578afd-1619-4731-af8a-4182b94c97f0',
                                'itemType': 'application/vnd.sas.models.variable',
                                'method': 'GET',
                                'rel': 'up',
                        

---